In [27]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
import os
from supabase import create_client, Client
import math
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [29]:

def get_filter(fh_q_dict, category, soma):
    soma = math.ceil(soma)
    if category == 'corners':
            atual = fh_q_dict[category]['sum_1st_cornerkicks'][2] 
            target = atual + soma
            response = (
            supabase.table(table)
            .select("*") 
            .eq('sum_period1', fh_q_dict[category]['sum_period1'])
            .eq('sum_1st_cornerkicks', fh_q_dict[category]['sum_1st_cornerkicks'][2])
            .gte('sum_all_cornerkicks', target)
            .gte("sum_1st_totalshotsongoal", fh_q_dict[category]['sum_1st_totalshotsongoal'][0]).lte("sum_1st_totalshotsongoal", fh_q_dict[category]['sum_1st_totalshotsongoal'][1])
            
            .execute()
        )
    elif category == 'gols':
        atual = fh_q_dict[category]['sum_period1']
        target = atual + soma
        response = (
        supabase.table(table)
        .select("*")
        .eq('sum_period1', fh_q_dict[category]['sum_period1'])
        .gte('sum_normaltime', target)
        .gte("sum_expected_goals_1st", fh_q_dict[category]['sum_expected_goals_1st'][0]).lte("sum_expected_goals_1st", fh_q_dict[category]['sum_expected_goals_1st'][1])
        .gte("sum_1st_shotsongoal", fh_q_dict[category]['sum_1st_shotsongoal'][0]).lte("sum_1st_shotsongoal", fh_q_dict[category]['sum_1st_shotsongoal'][1])
        .gte("sum_1st_totalshotsinsidebox", fh_q_dict[category]['sum_1st_totalshotsinsidebox'][0]).lte("sum_1st_totalshotsinsidebox", fh_q_dict[category]['sum_1st_totalshotsinsidebox'][1])
        .gte("sum_1st_finalthirdentries", fh_q_dict[category]['sum_1st_finalthirdentries'][0]).lte("sum_1st_finalthirdentries", fh_q_dict[category]['sum_1st_finalthirdentries'][1])
        
        .execute()
        )

    elif category == 'yellowcards':
        atual = fh_q_dict[category]['sum_1st_yellowcards'][2]
        target = atual + soma
        response = (
        supabase.table(table)
        .select("*")
        .eq('sum_1st_yellowcards', fh_q_dict[category]['sum_1st_yellowcards'][2])
        .gte('sum_all_yellowcards', target)                     
        .gte("sum_1st_freekicks", fh_q_dict[category]['sum_1st_freekicks'][0]).lte("sum_1st_freekicks", fh_q_dict[category]['sum_1st_freekicks'][1])
        .gte("sum_1st_finalthirdentries", fh_q_dict[category]['sum_1st_finalthirdentries'][0]).lte("sum_1st_finalthirdentries", fh_q_dict[category]['sum_1st_finalthirdentries'][1])
        
        .execute()
    )

    return len(response.data)

In [72]:

event_id = 14064418
dict_campo_amostral, fh_q_dict = campo_amostral(event_id)
# dict_campo_amostral, fh_q_dict

In [73]:
dict_campo_amostral

{'gols': 145, 'corners': 171, 'yellowcards': 576}

In [74]:
category = 'corners'
soma = 7
get_filter(fh_q_dict, category, soma)

49

In [75]:
fh_q_dict

{'gols': {'sum_period1': 0,
  'sum_expected_goals_1st': (0.14, 1.1400000000000001),
  'sum_1st_totalshotsongoal': (9, 15),
  'sum_1st_shotsongoal': (3, 7),
  'sum_1st_totalshotsinsidebox': (8, 12),
  'sum_1st_finalthirdentries': (25, 75),
  'sum_1st_cornerkicks': (1, 7, 4),
  'sum_1st_freekicks': (10, 16),
  'sum_1st_yellowcards': (0, 4, 2)},
 'corners': {'sum_period1': 0,
  'sum_expected_goals_1st': (0.39, 0.89),
  'sum_1st_totalshotsongoal': (10, 14),
  'sum_1st_shotsongoal': (4, 6),
  'sum_1st_totalshotsinsidebox': (9, 11),
  'sum_1st_finalthirdentries': (35, 65),
  'sum_1st_cornerkicks': (2, 6, 4),
  'sum_1st_freekicks': (11, 15),
  'sum_1st_yellowcards': (1, 3, 2)},
 'yellowcards': {'sum_period1': 0,
  'sum_expected_goals_1st': (0.39, 0.89),
  'sum_1st_totalshotsongoal': (10, 14),
  'sum_1st_shotsongoal': (4, 6),
  'sum_1st_totalshotsinsidebox': (9, 11),
  'sum_1st_finalthirdentries': (35, 65),
  'sum_1st_cornerkicks': (2, 6, 4),
  'sum_1st_freekicks': (11, 15),
  'sum_1st_yellowc

In [82]:
print(dict_campo_amostral['gols'])
print(get_filter(fh_q_dict, 'gols', 1))

145


118


In [112]:
def mount_gols_filters(fh_q_dict, dict_campo_amostral):
    category = 'gols'
    gol_atual = fh_q_dict['gols']['sum_period1']
    ca_category = dict_campo_amostral['gols'] 
    print(gol_atual)
    dict_gols_filter_over = {
        1 :  [gol_atual + 0.5, get_filter(fh_q_dict, category, 1),0,0],
        2 : [gol_atual + 1.5, get_filter(fh_q_dict, category, 2), 0,0],
        3 : [gol_atual + 2.5, get_filter(fh_q_dict, category, 3), 0,0]
    }

    dict_gols_filter_under = {
        1 : [gol_atual + 0.5 ,ca_category - get_filter(fh_q_dict, category, 1), 0, 0],
        2 : [gol_atual + 1.5, ca_category - get_filter(fh_q_dict, category, 2),0, 0],
        3 : [gol_atual + 2.5, ca_category - get_filter(fh_q_dict, category, 3),0, 0]
    }

    print(ca_category)
    display(dict_gols_filter_over)
    display(dict_gols_filter_under)

    return dict_gols_filter_over, dict_gols_filter_under


In [113]:
dict_gols_filter_over, dict_gols_filter_under = mount_gols_filters(fh_q_dict, dict_campo_amostral)

0
145


{1: [0.5, 118, 0, 0], 2: [1.5, 64, 0, 0], 3: [2.5, 32, 0, 0]}

{1: [0.5, 27, 0, 0], 2: [1.5, 81, 0, 0], 3: [2.5, 113, 0, 0]}

In [114]:
dict_gols_filter_under

{1: [0.5, 27, 0, 0], 2: [1.5, 81, 0, 0], 3: [2.5, 113, 0, 0]}

In [115]:
prob = (dict_gols_filter_over[i][1] * 100) / dict_campo_amostral['gols']
prob


22.06896551724138

In [120]:
def mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under):
    
    for i in range(1,len(dict_gols_filter_over) + 1):
        filter = dict_gols_filter_over[i][1]
        prob = (filter * 100) / dict_campo_amostral['gols']
        dict_gols_filter_over[i][2] = prob


    for i in range(1,len(dict_gols_filter_under) + 1):
        filter = dict_gols_filter_under[i][1]
        prob = (filter * 100) / dict_campo_amostral['gols']
        dict_gols_filter_under[i][2] = prob

    return dict_gols_filter_over, dict_gols_filter_under

mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under)

({1: [0.5, 118, 81.37931034482759, 0],
  2: [1.5, 64, 44.13793103448276, 0],
  3: [2.5, 32, 22.06896551724138, 0]},
 {1: [0.5, 27, 18.620689655172413, 0],
  2: [1.5, 81, 55.86206896551724, 0],
  3: [2.5, 113, 77.93103448275862, 0]})

In [121]:
def mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under):
    for i in range(1,len(dict_gols_filter_over) + 1):
        prob = (dict_gols_filter_over[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_over[i][2] = prob

    for i in range(1,len(dict_gols_filter_under) + 1):
        prob = (dict_gols_filter_under[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_under[i][2] = prob

    return dict_gols_filter_over, dict_gols_filter_under

mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under)

({1: [0.5, 118, 81.37931034482759, 0],
  2: [1.5, 64, 44.13793103448276, 0],
  3: [2.5, 32, 22.06896551724138, 0]},
 {1: [0.5, 27, 18.620689655172413, 0],
  2: [1.5, 81, 55.86206896551724, 0],
  3: [2.5, 113, 77.93103448275862, 0]})